(d_raster_crs_intro)=

# Raster CRS 
 
In this section we will learn how to 'reproject' or warp raster data.

Learning Objectives:
- What is warping?
- What is an affine transformation?


:::{warning}
This section is heavily dependent on the lesson about affine transformations. Please review this content [here](d_affine).
:::
 
## Warping Images